In [1]:
reddit_id = '<KEY>'
reddit_secret = '<KEY>'
aws_key = '<KEY>'
aws_secret = '<KEY>'
bucket = '<KEY>'

In [2]:
!pip install praw

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.0/191.0 kB 4.9 MB/s eta 0:00:00


In [3]:
import praw

reddit = praw.Reddit(
    client_id= reddit_id ,
    client_secret= reddit_secret,
    user_agent="my user agent",
    check_for_async=False
)

In [11]:
from datetime import datetime
import pandas as pd
from io import StringIO

In [5]:
!pip install boto3

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 76.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 10.1 MB/s eta 0:00:00


In [6]:
import boto3

BUCKET_NAME = bucket # replace with your bucket name

# enter authentication credentials
s3 = boto3.resource('s3', aws_access_key_id = aws_key,
                          aws_secret_access_key= aws_secret )

In [12]:
dfOut = pd.DataFrame(columns =['id','dt','author','body'])
new = pd.DataFrame(columns =['id','dt','author','body'])

subreddit = reddit.subreddit("Singapore")
for comment in subreddit.stream.comments(skip_existing=True):
  new.loc[0,'id'] = comment.id
  new.loc[0,"dt"] = datetime.utcfromtimestamp(comment.created_utc)
  new.loc[0,'body'] = comment.body
  new.loc[0,'author'] = comment.author
  print(new)

  dfOut = pd.concat([dfOut,new]).reset_index(drop = True)


  if len (dfOut) % 10 == 0:


    latest_dt = new.loc[0,'dt']
    filename =f'Out_{latest_dt}.csv'
    csv_buffer = StringIO()
    dfOut.to_csv(csv_buffer, index= False)
    s3.Bucket(BUCKET_NAME).put_object(Key= filename, Body=csv_buffer.getvalue())
    dfOut = dfOut[0:0]

### for saving to local ####
    # dfOut.to_csv(f'Out_{latest_dt}.csv')
    # dfOut = dfOut[0:0]


        id                   dt         author  \
0  kpkmfxu  2024-02-09 01:31:09  Jumpstart_411   

                                                body  
0  Increase GST, vouchers and subsidies.  The inf...  
        id                   dt        author  \
0  kpkmoaz  2024-02-09 01:32:45  anangrypudge   

                                                body  
0  \nDon’t wait so long to make more 龙\n\nWait to...  
        id                   dt              author                   body
0  kpkmpjr  2024-02-09 01:33:00  Interesting_Ad2986  Typical chinaman, no?
        id                   dt          author  \
0  kpkmryw  2024-02-09 01:33:29  freshcheesepie   

                                                body  
0  Just curious, none of the 4th gen lee come out...  
        id                   dt   author  \
0  kpkms3l  2024-02-09 01:33:30  doc-tom   

                                                body  
0  >You are right that the government has never s...  
        id        

KeyboardInterrupt: 